<a href="https://colab.research.google.com/github/rizkyabdillah127/transformer_translation/blob/main/Eksplorasi_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tugas Eksplorasi Transformer

122140127

122140165

122140145


Tahapan Awal Yang Kami Lakukan Adalah

Menginstal atau memperbarui spaCy, yaitu library NLP (Natural Language Processing) yang digunakan untuk tokenisasi, lemmatization, dan pemrosesan teks lainnya.

Mengunduh model bahasa Inggris kecil (small) untuk spaCy.
Model ini berisi kamus, aturan grammar, dan vektor kata untuk bahasa Inggris.

Mengunduh model bahasa Prancis kecil (small) untuk spaCy.
Digunakan agar spaCy bisa memproses teks berbahasa Prancis dengan benar (misalnya untuk tokenisasi).

Menginstal PyTorch versi 2.3.0 dan TorchText versi 0.18.0,
dua library utama untuk membuat dan melatih model deep learning (Transformer) dan memproses teks secara efisien.

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
!pip install torch==2.3.0 torchtext==0.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 113.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 120.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Bagian kode ini adalah tahap **impor pustaka (library)**, yang berfungsi seperti "mempersiapkan kotak peralatan" sebelum Anda memulai proyek NLP. Pertama, Anda mengimpor **PyTorch** (`torch`) sebagai mesin *deep learning* utama, beserta modul-modul intinya: `torch.nn` (untuk membangun arsitektur model seperti *layers* dan *loss functions*), `torch.optim` (untuk algoritma pelatihan seperti Adam/SGD), serta `DataLoader` dan `Dataset` (untuk mengelola dan memuat data Anda secara efisien). Khusus untuk NLP, Anda mengimpor `pad_sequence` untuk menyeragamkan panjang kalimat yang berbeda-beda. Selanjutnya, Anda mengimpor alat bantu pra-pemrosesan data: **Pandas** (`pd`) untuk memuat dan memanipulasi data (misalnya dari file CSV), **SpaCy** untuk melakukan tokenisasi (memecah teks menjadi kata), dan **TorchText** untuk membuat kosakata (mengubah kata menjadi angka). Terakhir, Anda mengimpor utilitas penting: `train_test_split` dari **Scikit-learn** (sklearn) untuk membagi data latih dan validasi (penting untuk mengecek performa model), serta pustaka standar Python (`math`, `time`, `os`, `random`) untuk operasi matematika, pengaturan waktu, manajemen file, dan memastikan hasil yang konsisten (reproducibility).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import spacy
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.vocab import build_vocab_from_iterator
from sklearn.model_selection import train_test_split # Penting untuk ValLoss/ValAcc

import math
import time
import os
import random

Selanjutnya ada perintah khusus untuk lingkungan **Google Colab**, yang berfungsi untuk **menghubungkan (me-"mount") Google Drive Anda** ke dalam *runtime* (mesin virtual) Colab.

Secara sederhana:

1.  **`from google.colab import drive`**: Mengimpor pustaka `drive` yang disediakan oleh Google Colab.
2.  **`drive.mount('/content/drive')`**: Memerintahkan Colab untuk "memasang" Google Drive Anda.

Setelah Anda menjalankan kode ini, Colab akan meminta izin Anda (biasanya lewat *pop-up*) untuk mengakses file di Google Drive Anda. Setelah Anda memberi izin, semua file dan folder di Google Drive Anda akan muncul dan bisa diakses dari dalam Colab melalui direktori `/content/drive`.

Ini sangat penting untuk:
* **Membaca dataset** yang telah Anda simpan di Drive.
* **Menyimpan model** yang sudah dilatih agar tidak hilang saat *runtime* Colab mati.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Bagian kode dibawah ini melakukan dua tugas penyiapan (setup) yang krusial sebelum memulai pelatihan model *deep learning*:

**1. memilih perangkat keras (hardware) terbaik**

 yang tersedia secara otomatis. Perintah `DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')` memeriksa apakah ada GPU NVIDIA (`'cuda'`) yang bisa digunakan. Jika ya, `DEVICE` akan diatur ke `'cuda'`, karena pelatihan *deep learning* jauh lebih cepat di GPU. Jika tidak ada GPU, `DEVICE` akan diatur ke `'cpu'`. Pesan konfirmasi kemudian dicetak untuk memberitahu Anda perangkat mana yang aktif.

 **2. mengatur "seed" untuk reproduktifitas (reproducibility)**.

 Dalam *deep learning*, banyak operasi (seperti inisialisasi bobot model atau pengacakan data) bersifat acak. Dengan menetapkan `SEED` (misalnya `1234`) dan memberikannya ke `random.seed`, `torch.manual_seed`, dan `torch.cuda.manual_seed`, Anda "mengunci" keacakan ini. Ditambah dengan `torch.backends.cudnn.deterministic = True`, ini memastikan bahwa setiap kali Anda menjalankan kode ini dari awal, Anda akan mendapatkan hasil yang *persis* sama, yang sangat penting untuk eksperimen yang valid dan perbandingan antar model.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Menggunakan device: {DEVICE}")

SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Menggunakan device: cuda


Ini adalah bagian **Persiapan Data (Data Preparation)**, salah satu tahap paling krusial dalam proyek *machine translation*.

Tujuan utama dari blok kode ini adalah untuk mengambil file teks mentah (Bahasa Inggris dan Prancis) dan mengubahnya menjadi format terstruktur yang siap "dimakan" oleh model *deep learning* (PyTorch).

Berikut adalah rincian langkah-langkahnya:

## 1. Memuat Tokenizer
* **Apa:** Kode ini memuat dua model bahasa Spacy yang telah dilatih sebelumnya (`en_core_web_sm` untuk Inggris dan `fr_core_news_sm` untuk Prancis).
* **Kenapa:** Kita memerlukan "tokenizer" untuk memecah kalimat mentah (string) menjadi daftar kata-kata individual (token). Contoh: `"hello world"` menjadi `['hello', 'world']`. Dua fungsi, `tokenize_en` dan `tokenize_fr`, dibuat untuk melakukan tugas ini.


## 2. Memuat dan Memvalidasi Dataset
* **Apa:** Kode ini mencoba membaca dua file teks (`.csv` yang diperlakukan sebagai teks biasa) dari Google Drive Anda. Satu file berisi kalimat-kalimat Bahasa Inggris dan satu lagi berisi terjemahannya dalam Bahasa Prancis.
* **Kenapa:** Ini adalah data mentah kita. Kode ini mengasumsikan bahwa data tersebut adalah *paralel*, artinya baris ke-5 di file Inggris adalah terjemahan dari baris ke-5 di file Prancis.
* **Penanganan Error:**
    * Jika file **tidak ditemukan**, kode ini secara cerdas membuat *dummy dataset* (data palsu) kecil agar sisa kode tetap bisa berjalan tanpa error.
    * Jika jumlah baris **tidak sama**, kode ini akan memotong dataset ke jumlah baris minimum agar tetap sinkron.
* **Output:** Sebuah **Pandas DataFrame** `df` dibuat, di mana setiap baris berisi satu pasang kalimat (`'en'` dan `'fr'`).


## 3. Pemisahan Data (Train & Validation)
* **Apa:** Dataset `df` dipecah menjadi dua bagian menggunakan `train_test_split`.
* **Kenapa:** Ini adalah praktik standar *machine learning*.
    * **Training Data (`train_df`):** Bagian terbesar (di sini 90%) yang digunakan untuk *mengajari* model.
    * **Validation Data (`val_df`):** Bagian kecil (10%) yang "disembunyikan" dari model selama pelatihan. Data ini dipakai untuk *menguji* seberapa baik model bekerja pada data yang belum pernah dilihatnya (untuk mengecek *Val Loss/Accuracy*).
* `random_state=SEED` digunakan agar hasil pemisahannya selalu sama setiap kali kode dijalankan.


## 4. Membangun Kosakata (Vocabulary)
* **Apa:** Kode ini membangun dua "kamus" (`vocab_src` untuk Inggris dan `vocab_tgt` untuk Prancis) dari data pelatihan.
* **Kenapa:** Model *deep learning* tidak mengerti kata "hello"; mereka hanya mengerti **angka**. *Vocabulary* ini memetakan setiap kata unik ke sebuah angka (indeks).
* **Detail Penting:**
    * **`min_freq=2`**: Kata yang hanya muncul sekali di data latih akan dibuang (dianggap sebagai `<unk>`). Ini membantu mengurangi ukuran *vocabulary* dan membuang kata-kata yang mungkin *typo*.
    * **`special_symbols`**: Empat token khusus ditambahkan:
        * `<unk>`: Untuk kata yang tidak dikenal (tidak ada di *vocabulary*).
        * `<pad>`: (Padding) Untuk "mengisi" kalimat yang lebih pendek agar sama panjangnya dalam satu *batch*.
        * `<bos>`: (Begin of Sentence) Tanda penanda awal kalimat.
        * `<eos>`: (End of Sentence) Tanda penanda akhir kalimat.


## 5. Membuat `Dataset` Kustom
* **Apa:** Sebuah `class` Python bernama `TranslationDataset` dibuat menggunakan `Dataset` milik PyTorch.
* **Kenapa:** Ini adalah "cetak biru" (blueprint) standar PyTorch untuk data Anda. Ini memberi tahu PyTorch cara mengambil **satu** item data.
* **Fungsi `__getitem__`**: Ini adalah intinya. Ketika diminta data di indeks `idx`, fungsi ini akan:
    1.  Mengambil kalimat 'en' dan 'fr'.
    2.  Melakukan tokenisasi (memecahnya jadi kata).
    3.  Mengubah kata-kata menjadi angka menggunakan *vocabulary* yang tadi dibuat.
    4.  Menambahkan token `<bos>` di awal dan `<eos>` di akhir.
    5.  Mengembalikan hasilnya sebagai `torch.tensor`.


## 6. Membuat `DataLoader`
* **Apa:** Kode ini membungkus `TranslationDataset` (untuk data latih dan validasi) dengan `DataLoader`.
* **Kenapa:** `DataLoader` adalah "mesin" yang secara otomatis mengambil data dari `Dataset` dan mengaturnya untuk pelatihan. Ia menangani:
    * **Batching**: Mengelompokkan beberapa kalimat (misalnya 64) menjadi satu *batch* untuk efisiensi GPU.
    * **Shuffling**: Mengacak urutan data latih di setiap *epoch* (penting agar model tidak "menghafal" urutan data).
    * **Collation (`collate_fn`)**: Ini sangat penting. Karena kalimat dalam satu *batch* punya panjang berbeda, fungsi `collate_fn` **menambahkan *padding* (`<pad>`)** ke kalimat yang lebih pendek sehingga semua kalimat dalam *batch* memiliki panjang yang sama. Ini mengubahnya menjadi *tensor* persegi yang rapi.
* **`batch_first=False`**: Ini adalah pengaturan penting yang berarti *tensor* yang dihasilkan akan memiliki bentuk `[panjang_kalimat, batch_size]`, format yang umum digunakan oleh model Transformer.

Singkatnya, **BAGIAN 1** ini mengubah dua file `.csv` mentah menjadi dua objek `DataLoader` (`train_dataloader` dan `val_dataloader`) yang sangat efisien dan siap dimasukkan ke dalam model PyTorch untuk pelatihan.

In [ ]:
# ----------------------------------------------------------------------
# BAGIAN 1: FOKUS PENILAIAN 1 - DATA PREPARATION (BOBOT 20%)
# ----------------------------------------------------------------------

print("\n--- Memulai Fokus Penilaian 1: Data Preparation ---")

# --- 1.1. Memuat Tokenizer Spacy ---
try:
    spacy_en = spacy.load('en_core_web_sm')
    spacy_fr = spacy.load('fr_core_news_sm') # <-- BERUBAH: Muat model Prancis
    print("Tokenizer Spacy (en/fr) berhasil dimuat.")
except IOError:
    print("Error: Model Spacy tidak ditemukan. Pastikan instalasi di atas berhasil.")
    exit()

# Fungsi tokenizer (Source: English)
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(str(text))]

# Fungsi tokenizer (Target: French)
def tokenize_fr(text): # <-- BERUBAH: Nama fungsi dan model
    return [tok.text for tok in spacy_fr.tokenizer(str(text))]

# --- 1.2. Memuat Dataset dari Google Drive (Format File Teks Paralel) ---
# PENTING: Pastikan kedua file ada di folder 'MyDrive' Anda.
EN_FILE_PATH = '/content/drive/MyDrive/DictionaryDataset/small_vocab_en.csv'
FR_FILE_PATH = '/content/drive/MyDrive/DictionaryDataset/small_vocab_fr.csv'

if not os.path.exists(EN_FILE_PATH) or not os.path.exists(FR_FILE_PATH):
    print(f"Error: Salah satu file tidak ditemukan.")
    print(f"Pastikan '{EN_FILE_PATH}' dan '{FR_FILE_PATH}' ada.")
    # Membuat data dummy agar kode tetap berjalan untuk demo
    df = pd.DataFrame({
        'en': ['hello world', 'how are you', 'good morning', 'i am a student'],
        'fr': ['bonjour le monde', 'comment vas-tu', 'bonjour', 'je suis étudiant']
    })
else:
    print(f"Memuat dataset dari: {EN_FILE_PATH} dan {FR_FILE_PATH}")

    # Baca file Inggris
    with open(EN_FILE_PATH, 'r', encoding='utf-8') as f:
        en_sentences = [line.strip() for line in f]

    # Baca file Prancis
    with open(FR_FILE_PATH, 'r', encoding='utf-8') as f:
        fr_sentences = [line.strip() for line in f]

    # Pastikan kedua file memiliki jumlah baris yang sama
    if len(en_sentences) != len(fr_sentences):
        print(f"Error: Jumlah baris tidak sama! EN: {len(en_sentences)}, FR: {len(fr_sentences)}")
        # (Lanjutkan dengan data yang dipotong agar demo tetap jalan)
        min_len = min(len(en_sentences), len(fr_sentences))
        en_sentences = en_sentences[:min_len]
        fr_sentences = fr_sentences[:min_len]

    # Buat DataFrame
    df = pd.DataFrame({'en': en_sentences, 'fr': fr_sentences})

print(f"Dataset berhasil dimuat. Jumlah total pasangan kalimat: {len(df)}")

# --- 1.3. Pemisahan Data (Train & Validation) ---
train_df, val_df = train_test_split(df, test_size=0.1, random_state=SEED)
print(f"Data dibagi: {len(train_df)} train, {len(val_df)} validasi.")

# --- 1.4. Pembangunan Vocabulary ---
# Dok: Karena ini 'small_vocab', kita bisa gunakan min_freq=2
def yield_tokens(data_iter, tokenizer):
    for text in data_iter:
        yield tokenizer(text)

# Token spesial
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# Vocabulary untuk Bahasa Inggris (Source)
vocab_src = build_vocab_from_iterator(
    yield_tokens(train_df['en'], tokenize_en),
    min_freq=2, # Kita set 2 untuk membuang kata yang sangat langka
    specials=special_symbols,
    special_first=True
)
vocab_src.set_default_index(UNK_IDX)

# Vocabulary untuk Bahasa Prancis (Target)
vocab_tgt = build_vocab_from_iterator(
    yield_tokens(train_df['fr'], tokenize_fr), # <-- BERUBAH: 'fr' dan tokenize_fr
    min_freq=2,
    specials=special_symbols,
    special_first=True
)
vocab_tgt.set_default_index(UNK_IDX)

SRC_VOCAB_SIZE = len(vocab_src)
TGT_VOCAB_SIZE = len(vocab_tgt)
print(f"Ukuran Vocab Source (EN): {SRC_VOCAB_SIZE}")
print(f"Ukuran Vocab Target (FR): {TGT_VOCAB_SIZE}") # <-- BERUBAH: FR

# --- 1.5. Class Kustom PyTorch Dataset ---
class TranslationDataset(Dataset):
    def __init__(self, df, vocab_src, vocab_tgt, tokenize_src, tokenize_tgt):
        self.df = df
        self.vocab_src = vocab_src
        self.vocab_tgt = vocab_tgt
        self.tokenize_src = tokenize_src
        self.tokenize_tgt = tokenize_tgt

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        src_text = row['en']
        tgt_text = row['fr'] # <-- BERUBAH: 'fr'

        src_tensor = torch.tensor([BOS_IDX] + [self.vocab_src[token] for token in self.tokenize_src(src_text)] + [EOS_IDX], dtype=torch.long)
        tgt_tensor = torch.tensor([BOS_IDX] + [self.vocab_tgt[token] for token in self.tokenize_tgt(tgt_text)] + [EOS_IDX], dtype=torch.long)
        return src_tensor, tgt_tensor

# --- 1.6. Collate Function dan DataLoader ---
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=False)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=False)
    return src_batch, tgt_batch

# MENGURANGI BATCH SIZE untuk membantu mencegah OOM Error
BATCH_SIZE = 64

# Buat instance Dataset
train_dataset = TranslationDataset(train_df, vocab_src, vocab_tgt, tokenize_en, tokenize_fr) # <-- BERUBAH
val_dataset = TranslationDataset(val_df, vocab_src, vocab_tgt, tokenize_en, tokenize_fr) # <-- BERUBAH

# Buat DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

print(f"DataLoader berhasil dibuat dengan Batch Size: {BATCH_SIZE}")
print("--- Selesai Fokus Penilaian 1 ---")


--- Memulai Fokus Penilaian 1: Data Preparation ---
Tokenizer Spacy (en/fr) berhasil dimuat.
Memuat dataset dari: /content/drive/MyDrive/DictionaryDataset/small_vocab_en.csv dan /content/drive/MyDrive/DictionaryDataset/small_vocab_fr.csv
Dataset berhasil dimuat. Jumlah total pasangan kalimat: 137860
Data dibagi: 124074 train, 13786 validasi.
Ukuran Vocab Source (EN): 205
Ukuran Vocab Target (FR): 338
DataLoader berhasil dibuat dengan Batch Size: 64
--- Selesai Fokus Penilaian 1 ---


**BAGIAN 2**, inti dari proyek ini, di mana Anda **membangun arsitektur model Transformer** itu sendiri. Anda tidak melatihnya di sini, tetapi Anda mendefinisikan "cetak biru" atau kerangka dari model yang akan melakukan penerjemahan.

Berikut adalah penjelasan dari setiap komponen utamanya:

## 1. Komponen Pendukung: `TokenEmbedding` dan `PositionalEncoding`

Sebelum membangun model utamanya, Anda perlu dua komponen penting:

* **`TokenEmbedding` (Kamus Makna)**: Ini adalah *layer* sederhana yang mengubah input Anda (yang masih berupa angka/indeks, misal: `[2, 56, 34]`) menjadi **vektor makna (embeddings)**. `nn.Embedding` bertindak seperti tabel pencarian (lookup table) raksasa di mana setiap angka indeks kata (misal, 56 untuk kata "hello") dipetakan ke vektor padat (misal, berukuran 512) yang merepresentasikan "makna" dari kata tersebut.
* **`PositionalEncoding` (GPS Urutan Kata)** : Ini adalah "trik" jenius dari arsitektur Transformer. Tidak seperti RNN, Transformer memproses semua kata dalam kalimat secara bersamaan (paralel), sehingga ia **tidak tahu urutan kata** (misal, bedanya "kucing makan tikus" vs "tikus makan kucing"). *Positional Encoding* membuat "vektor posisi" yang unik untuk setiap urutan (posisi 1, 2, 3, dst.) menggunakan fungsi matematika `sin` dan `cos`. Vektor posisi ini kemudian **ditambahkan** ke vektor makna (`TokenEmbedding`) untuk memberikan model informasi kontekstual tentang di mana setiap kata berada dalam kalimat.


## 2. Model Utama: `Seq2SeqTransformer`

Ini adalah kelas utama yang menggabungkan semua bagian menjadi satu model utuh.

* **`__init__` (Inisialisasi)**: Di sinilah Anda "merakit pabriknya".
    1.  Anda memanggil `nn.Transformer` bawaan PyTorch, yang merupakan inti modelnya (berisi tumpukan Encoder dan Decoder). Anda memberinya parameter penting seperti `d_model` (ukuran embedding), `nhead` (jumlah *attention heads*), dan jumlah *layers*.
    2.  Anda membuat dua `TokenEmbedding`: satu untuk *source* (Inggris) dan satu untuk *target* (Prancis).
    3.  Anda membuat satu `PositionalEncoding`.
    4.  Anda membuat `nn.Linear` layer terakhir yang disebut `generator`. Tugasnya adalah mengambil output akhir dari Transformer (sebuah vektor) dan mengubahnya menjadi skor probabilitas untuk *setiap kata* dalam kosakata target (Prancis).

* **`forward` (Alur Pelatihan)**: Fungsi ini mendefinisikan apa yang terjadi ketika Anda memberi model data **selama pelatihan**.
    1.  Ambil kalimat *source* (src) dan *target* (trg).
    2.  Ubah keduanya menjadi *embeddings* (vektor makna).
    3.  Tambahkan *positional encoding* ke kedua embeddings tersebut.
    4.  Masukkan embeddings yang sudah lengkap ini, beserta *masking* (lihat poin 3), ke dalam `nn.Transformer`.
    5.  Ambil output Transformer dan masukkan ke `generator` untuk mendapatkan prediksi akhir.

* **`encode` & `decode`**: Ini adalah fungsi terpisah yang biasanya digunakan saat **inferensi** (menerjemahkan kalimat baru), di mana Anda hanya perlu menjalankan Encoder satu kali (`encode`) dan Decoder berulang kali (`decode`) kata per kata.


## 3. Aturan Main: Fungsi Masking (`create_mask`)

Ini adalah bagian yang sangat penting dan seringkali rumit. *Masking* adalah "aturan main" yang memberi tahu model bagian mana dari data yang boleh (atau tidak boleh) dilihat.

* **`src_padding_mask` & `tgt_padding_mask` (Masking Padding)**: Ingat di Bagian 1 kita menambahkan token `<pad>` untuk membuat semua kalimat sama panjang? Masking ini memberi tahu model, "Hei, token `<pad>` ini tidak penting. **Abaikan mereka** saat melakukan perhitungan *self-attention*." Ini mencegah model "belajar" dari padding buatan.

* **`tgt_mask` (Subsequent Mask / Look-Ahead Mask)**: Ini adalah topeng terpenting untuk *Decoder*. Saat melatih model untuk menerjemahkan "je suis étudiant", dan model sedang mencoba memprediksi kata "suis" (kata ke-2), ia **TIDAK BOLEH** "curang" dan melihat jawaban yang benar ("étudiant", kata ke-3). Masking ini berbentuk segitiga yang **menyembunyikan semua kata di masa depan** dari posisi saat ini. Ini memaksa model untuk belajar memprediksi kata *selanjutnya* hanya berdasarkan kata *sebelumnya*.

In [ ]:
# ----------------------------------------------------------------------
# BAGIAN 2: FOKUS PENILAIAN 2 - ARSITEKTUR TRANSFORMER (BOBOT 25%)
# ----------------------------------------------------------------------

print("\n--- Memulai Fokus Penilaian 2: Definisi Arsitektur ---")

# --- 2.1. Class PositionalEncoding ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# --- 2.2. Class TokenEmbedding ---
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# --- 2.3. Class Seq2SeqTransformer (Model Utama) ---
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, emb_size, nhead,
                 src_vocab_size, tgt_vocab_size, dim_feedforward=512, dropout=0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(self, src, trg, src_mask, tgt_mask,
                src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src_emb, tgt_emb, src_mask, tgt_mask, None,
            src_padding_mask, tgt_padding_mask, memory_key_padding_mask
        )
        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )

# --- 2.4. Fungsi Pembuatan Masking ---
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

print("Class dan fungsi arsitektur berhasil didefinisikan.")
print("--- Selesai Fokus Penilaian 2 ---")


--- Memulai Fokus Penilaian 2: Definisi Arsitektur ---
Class dan fungsi arsitektur berhasil didefinisikan.
--- Selesai Fokus Penilaian 2 ---


Ini adalah **BAGIAN 3**, "ruang mesin"  dari proyek Anda, di mana proses **pelatihan model** yang sebenarnya terjadi. Setelah data (Bagian 1) dan arsitektur (Bagian 2) siap, di sinilah model Anda "belajar" dari data.

Berikut adalah rincian prosesnya:

## 1. Inisialisasi (Persiapan Pelatihan)

Sebelum pelatihan dimulai, Anda perlu menyiapkan tiga komponen utama:

1.  **Model**: Anda membuat instansi dari kelas `Seq2SeqTransformer` yang Anda definisikan di Bagian 2.
    * **Penting**: Anda sengaja menggunakan *hyperparameter* yang **lebih kecil** (misalnya `EMB_SIZE = 256`, `FFN_HID_DIM = 256`). Ini adalah langkah praktis yang cerdas untuk **mengatasi error OOM (Out of Memory)**, terutama di lingkungan dengan VRAM terbatas seperti Colab. Model yang lebih ringan membutuhkan lebih sedikit memori.
    * Model kemudian "diterapkan" (`.apply(init_weights)`) dengan inisialisasi bobot (weights) yang baik (menggunakan *Xavier* dan *Normal distribution*) untuk membantu pelatihan berjalan lebih stabil.
    * Terakhir, model dipindahkan ke `DEVICE` (`.to(DEVICE)`) agar berjalan di GPU (`'cuda'`) jika tersedia, yang jauh lebih cepat.

2.  **Loss Function (Fungsi Kerugian)**: Anda mendefinisikan "juri" atau "penilai" untuk model.
    * `nn.CrossEntropyLoss`: Ini adalah pilihan standar untuk tugas klasifikasi (dan memprediksi kata berikutnya pada dasarnya adalah klasifikasi di antara semua kata di *vocabulary*).
    * `ignore_index=PAD_IDX`: Ini adalah pengaturan yang **sangat penting**. Ini memberi tahu "juri" untuk **mengabaikan** token `<pad>` saat menghitung *error*. Ini logis, karena kita tidak peduli jika model salah memprediksi *padding*; kita hanya peduli pada kata-kata yang sebenarnya.

3.  **Optimizer (Pengoptimal)**: Anda memilih "mekanik" yang akan memperbarui bobot model.
    * `optim.Adam`: Ini adalah algoritma optimasi yang paling umum digunakan, dikenal cepat dan efisien. Ia mengambil *error* (loss) yang dihitung oleh "juri" dan menggunakannya untuk menyesuaikan *setiap* parameter di dalam model secara cerdas agar kinerjanya menjadi sedikit lebih baik.

## 2. Fungsi Evaluasi (Kartu Rapor Model)

Anda mendefinisikan fungsi `evaluate` terpisah yang bertujuan untuk **mengukur performa model pada data yang belum pernah dilihatnya** (data validasi).

* `model.eval()`: Mengubah model ke mode "evaluasi", yang menonaktifkan fitur-fitur yang hanya dipakai saat latihan (seperti *dropout*).
* `with torch.no_grad()`: Perintah ini **mematikan perhitungan gradien**. Ini sangat penting karena saat evaluasi, Anda tidak perlu menghitung "siapa yang salah" (gradien), Anda hanya ingin tahu hasilnya. Ini menghemat banyak memori dan mempercepat proses.
* **Perhitungan Akurasi**: Selain menghitung *loss* (seberapa salah), fungsi ini juga menghitung *accuracy* (seberapa sering tebakannya benar). Ia melakukannya dengan mengambil prediksi (`.argmax()`), membandingkannya dengan jawaban sebenarnya, dan **hanya menghitung token yang bukan `<pad>`**.

## 3. Loop Pelatihan Utama (Proses Belajar)

Ini adalah inti dari proses belajar, yang diatur untuk berjalan hanya **1 epoch** (satu kali melihat keseluruhan data latih).

Loop ini terdiri dari dua fase:

### Fase 1: Training (Latihan)
Model diatur ke `model.train()` (mengaktifkan kembali *dropout*, dll.). Kemudian, ia mengulangi setiap *batch* data dari `train_dataloader`. Untuk setiap *batch*:

1.  **Persiapan Data**: Input target dipecah menjadi `tgt_input` (apa yang "dilihat" *decoder*, misal: `[<bos>, "je", "suis"]`) dan `tgt_out` (apa yang harus "ditebak" *decoder*, misal: `["je", "suis", <eos>]`). Ini disebut *teacher forcing*.
2.  **Forward Pass**: Data (`src` dan `tgt_input`) dan *masking* dimasukkan ke model (`logits = model(...)`). Model membuat prediksi (logits) untuk setiap kata dalam *batch*.
3.  **Hitung Loss**: `loss = loss_fn(...)` dihitung dengan membandingkan prediksi model (`logits`) dengan jawaban yang benar (`tgt_out`).
4.  **Backward Pass (Belajar)**:
    * `optimizer.zero_grad()`: Menghapus sisa perhitungan dari *batch* sebelumnya.
    * `loss.backward()`: Menghitung gradien, atau "menyebarkan kesalahan" ke seluruh jaringan untuk mencari tahu parameter mana yang paling berkontribusi pada kesalahan.
    * `optimizer.step()`: "Mekanik" (Adam) menggunakan gradien tadi untuk **memperbarui bobot (weights)** model. Di sinilah proses "belajar" yang sesungguhnya terjadi.

Setiap 50 *batch*, *loss* sementara dicetak agar Anda bisa memantau kemajuan.

### Fase 2: Validation (Validasi)
**Setelah** semua *batch* latihan dalam satu epoch selesai, kode ini:

1.  **Memanggil `evaluate(...)`**: Menjalankan fungsi "kartu rapor" yang Anda buat tadi pada **seluruh data validasi** (`val_dataloader`).
2.  **Mencetak Laporan**: Ini mencetak ringkasan kinerja akhir untuk epoch tersebut, yang berisi:
    * `TrainLoss`: Rata-rata *error* pada data yang dilihatnya saat latihan.
    * `ValLoss`: Rata-rata *error* pada data "ujian" (validasi).
    * `ValAcc`: Akurasi (persentase tebakan benar) pada data "ujian".

Ini adalah metrik terpenting untuk mengetahui apakah model Anda benar-benar belajar atau hanya menghafal.

In [ ]:
# ----------------------------------------------------------------------
# BAGIAN 3: FOKUS PENILAIAN 3 - PROSES PELATIHAN (BOBOT 35%)
# ----------------------------------------------------------------------

print("\n--- Memulai Fokus Penilaian 3: Proses Pelatihan ---")

# --- 3.1. Inisialisasi Model, Loss, dan Optimizer ---

# --- PERUBAHAN HYPERPARAMETER UNTUK MENGATASI OOM ---
# Kita gunakan arsitektur yang lebih kecil (ringan)
EMB_SIZE = 256       # (dari 512)
NHEAD = 8            # (256 % 8 == 0, jadi ini OK)
FFN_HID_DIM = 256    # (dari 512)
NUM_ENCODER_LAYERS = 3 # (dari 3, ini OK)
NUM_DECODER_LAYERS = 3 # (dari 3, ini OK)
LEARNING_RATE = 0.0001
DROPOUT = 0.1

# Hyperparameter Training (Sesuai Permintaan)
NUM_EPOCHS = 1 # Sesuai permintaan, hanya 1 epoch

# Inisialisasi model
model = Seq2SeqTransformer(
    NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD,
    SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, DROPOUT
).to(DEVICE)

# Inisialisasi bobot
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Embedding):
        nn.init.normal_(m.weight, mean=0, std=0.02)
model.apply(init_weights)

# Loss function
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.98), eps=1e-9)

print("Model (arsitektur lebih ringan), Loss, dan Optimizer berhasil diinisialisasi.")

# --- 3.2. Fungsi Evaluasi (Untuk ValLoss & ValAcc) ---
def evaluate(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for src, tgt in dataloader:
            src = src.to(DEVICE)
            tgt = tgt.to(DEVICE)

            tgt_input = tgt[:-1, :]
            tgt_out = tgt[1:, :]

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

            logits = model(
                src, tgt_input,
                src_mask, tgt_mask,
                src_padding_mask, tgt_padding_mask,
                src_padding_mask
            )

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            total_loss += loss.item()

            # --- Hitung Akurasi ---
            pred = logits.argmax(dim=-1)
            non_pad_mask = (tgt_out != PAD_IDX)
            correct = (pred == tgt_out) & non_pad_mask

            total_correct += correct.sum().item()
            total_tokens += non_pad_mask.sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_accuracy = total_correct / total_tokens if total_tokens > 0 else 0

    return avg_loss, avg_accuracy


# ----------------------------------------------------------------------
# BAGIAN 3.3: Loop Training Utama (MODIFIKASI - Validasi di Akhir Epoch)
# ----------------------------------------------------------------------

print("\n--- Memulai Loop Training (1 Epoch) ---")
print(f"Total batch per epoch: {len(train_dataloader)}")
# Hapus peringatan validasi per batch

start_time_epoch = time.time()

for epoch in range(NUM_EPOCHS):

    model.train() # Set model ke mode training
    epoch_loss = 0 # Kita akan kumpulkan total loss di sini

    print(f"\n--- Memulai Epoch {epoch+1}/{NUM_EPOCHS} ---")

    # 1. LOOP TRAINING BATCH
    for i, (src, tgt) in enumerate(train_dataloader):
        start_time_batch = time.time()

        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]
        tgt_out = tgt[1:, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(
            src, tgt_input,
            src_mask, tgt_mask,
            src_padding_mask, tgt_padding_mask,
            src_padding_mask
        )

        optimizer.zero_grad()
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        train_loss_batch = loss.item()
        epoch_loss += train_loss_batch

        # --- Modifikasi ---
        # Kita tidak lagi menjalankan validasi di sini.
        # Kita hanya akan print loss training sesekali agar terlihat progress.
        if (i + 1) % 50 == 0: # Print progress setiap 50 batch
             print(f"  Epoch {epoch+1} | Batch {i+1}/{len(train_dataloader)} | TrainLoss (batch): {train_loss_batch:.4f}")

    # -----------------------------------------------------------------
    # --- DOKUMENTASI PROSES PELATIHAN (SESUAI PERMINTAAN) ---
    # -----------------------------------------------------------------
    # 2. LOOP VALIDASI (Jalankan SETELAH epoch selesai)

    # Hitung rata-rata TrainLoss untuk epoch ini
    avg_train_loss = epoch_loss / len(train_dataloader)

    print("\n--- Selesai Epoch, Menjalankan Validasi ---")
    start_time_val = time.time()

    # Panggil fungsi evaluate satu kali
    val_loss, val_acc = evaluate(model, val_dataloader, loss_fn)

    val_time = time.time() - start_time_val

    # Tampilkan semua metrik di akhir epoch
    print(f"--- Laporan Akhir Epoch {epoch+1} ---")
    print(f"Waktu Validasi: {val_time:.2f} detik")
    print(f"  TrainLoss (Rata-rata): {avg_train_loss:.4f}")
    print(f"  ValLoss (Epoch): {val_loss:.4f}")
    print(f"  ValAcc  (Epoch): {val_acc*100:.2f}%")
    print("-" * 40)


# --- Laporan Akhir Training ---
end_time_epoch = time.time()
epoch_mins = (end_time_epoch - start_time_epoch) / 60

print("\n--- Selesai Fokus Penilaian 3 ---")
print(f"Total Waktu Training ({NUM_EPOCHS} Epoch): {epoch_mins:.2f} menit")
# Laporan final sudah dicetak di dalam loop di atas


--- Memulai Fokus Penilaian 3: Proses Pelatihan ---
Model (arsitektur lebih ringan), Loss, dan Optimizer berhasil diinisialisasi.

--- Memulai Loop Training (1 Epoch) ---
Total batch per epoch: 1939

--- Memulai Epoch 1/1 ---
  Epoch 1 | Batch 50/1939 | TrainLoss (batch): 3.2297
  Epoch 1 | Batch 100/1939 | TrainLoss (batch): 2.1783
  Epoch 1 | Batch 150/1939 | TrainLoss (batch): 1.5632
  Epoch 1 | Batch 200/1939 | TrainLoss (batch): 1.3448
  Epoch 1 | Batch 250/1939 | TrainLoss (batch): 1.1033
  Epoch 1 | Batch 300/1939 | TrainLoss (batch): 0.9203
  Epoch 1 | Batch 350/1939 | TrainLoss (batch): 0.7739
  Epoch 1 | Batch 400/1939 | TrainLoss (batch): 0.7295
  Epoch 1 | Batch 450/1939 | TrainLoss (batch): 0.5497
  Epoch 1 | Batch 500/1939 | TrainLoss (batch): 0.4479
  Epoch 1 | Batch 550/1939 | TrainLoss (batch): 0.3861
  Epoch 1 | Batch 600/1939 | TrainLoss (batch): 0.3129
  Epoch 1 | Batch 650/1939 | TrainLoss (batch): 0.2763
  Epoch 1 | Batch 700/1939 | TrainLoss (batch): 0.3492
  Ep

Ini adalah **BAGIAN 4**, tahap akhir yang disebut **Inferensi (Penerapan)**.

Setelah menyiapkan data (Bagian 1), merancang arsitektur (Bagian 2), dan melatih model (Bagian 3), kini saatnya **menggunakan model yang sudah dilatih** tersebut untuk melakukan tugas sebenarnya: menerjemahkan kalimat baru.

Berikut adalah rincian dari proses ini:

## 1. Fungsi `greedy_decode` (Inti Penerjemah)

Ini adalah fungsi "inti" yang menghasilkan terjemahan kata per kata. Prosesnya berbeda dengan saat pelatihan:

1.  **Encoder Dulu**: Model mengambil kalimat *source* (Inggris) dan menjalankannya melalui **Encoder** *hanya satu kali*. Hasilnya adalah `memory`, sebuah representasi makna dari kalimat sumber.
2.  **Decoder Mulai**: *Decoder* mulai dengan satu token, yaitu `<bos>` (awal kalimat).
3.  **Loop Generasi**: Model masuk ke dalam *loop* untuk menghasilkan kalimat *target* (Prancis):
    * Ia mengambil `memory` (dari Inggris) dan kalimat Prancis yang sudah dihasilkan *sejauh ini* (awalnya hanya `<bos>`).
    * Ia bertanya pada **Decoder**: "Berdasarkan ini, apa kata *selanjutnya*?"
    * Model memberikan probabilitas untuk setiap kata di *vocabulary* Prancis.
    * **"Greedy"**: Fungsi ini serakah. Ia tidak berpikir panjang; ia langsung memilih **satu kata dengan probabilitas tertinggi** (menggunakan `torch.max`).
    * Ia menambahkan kata baru ini ke kalimat yang sedang dibuat.
4.  **Berhenti**: Loop ini berlanjut (kata baru ditambahkan, lalu dipakai untuk memprediksi kata *berikutnya*, dan seterusnya) sampai model memprediksi token `<eos>` (akhir kalimat) atau mencapai batas panjang maksimum.

## 2. Fungsi `translate` (Pembungkus yang Ramah)

Fungsi `greedy_decode` bekerja dengan angka (tensor). Fungsi `translate` adalah "pembungkus" (wrapper) yang membuatnya **mudah digunakan oleh manusia**.

Fungsi ini menangani seluruh proses:

1.  **Pre-processing (Input)**: Mengambil kalimat mentah Bahasa Inggris (string), mengubahnya menjadi token, lalu mengubah token menjadi angka (indeks *vocabulary*), dan menambah token `<bos>`/`<eos>`.
2.  **Menjalankan Model**: Memanggil fungsi `greedy_decode` (inti penerjemah di atas) untuk mendapatkan hasil terjemahan dalam bentuk angka/tensor.
3.  **Post-processing (Output)**: Mengambil tensor angka dari model, mengubahnya kembali menjadi kata-kata (menggunakan *vocabulary* target/Prancis), dan membersihkan token `<bos>`/`<eos>` agar menjadi kalimat yang rapi dan bisa dibaca manusia.

## 3. Eksekusi Contoh (Test Drive)

Ini adalah bagian "test drive" di mana Anda akhirnya mencoba model Anda:

* **Mode `.eval()`**: Model diatur ke mode evaluasi, yang menonaktifkan *dropout* dan fitur pelatihan lainnya untuk hasil yang konsisten.
* **Contoh Validasi**: Kode ini mengambil dua contoh dari *validation set* (data yang tidak pernah dilihat model saat latihan). Ini adalah tes yang adil. Ia mencetak kalimat Inggris (Source), terjemahan yang benar (Target), dan terjemahan hasil tebakan model (Model).
* **Contoh Kustom**: Kode ini juga mencoba menerjemahkan kalimat yang benar-benar baru ("he is never cold in december .") untuk melihat bagaimana model menangani input yang tidak terduga.

**Peringatan Penting**: Seperti yang dicatat dalam kode, karena model **hanya dilatih 1 epoch** (untuk menghemat waktu dan memori), hasil terjemahannya kemungkinan besar **masih akan buruk** atau tidak masuk akal. Namun, yang terpenting adalah **proses inferensinya berjalan dengan benar** dari awal hingga akhir.

In [ ]:
# ----------------------------------------------------------------------
# BAGIAN 4: FOKUS PENILAIAN 4 - PENERAPAN (INFERENSI) (BOBOT 20%)
# ----------------------------------------------------------------------

print("\n--- Memulai Fokus Penilaian 4: Inferensi ---")

# --- 4.1. Fungsi Greedy Decode ---
# (Tidak ada perubahan, fungsi ini generik)
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    memory = model.encode(src, src_mask)
    memory = memory.to(DEVICE)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)

    for i in range(max_len - 1):
        tgt_seq_len = ys.size(0)
        tgt_mask = (generate_square_subsequent_mask(tgt_seq_len)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# --- 4.2. Fungsi Wrapper 'translate' ---
# (Tidak ada perubahan, fungsi ini generik)
def translate(model, src_sentence, vocab_src, vocab_tgt, tokenize_src):
    model.eval()
    tokens = [BOS_IDX] + [vocab_src[token] for token in tokenize_src(src_sentence)] + [EOS_IDX]
    src = torch.LongTensor(tokens).unsqueeze(1).to(DEVICE)
    src_mask = (torch.zeros(src.shape[0], src.shape[0])).type(torch.bool).to(DEVICE)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=50, start_symbol=BOS_IDX)
    tgt_words = [vocab_tgt.get_itos()[tok] for tok in tgt_tokens.flatten()]
    return " ".join(tgt_words).replace("<bos>", "").replace("<eos>", "").strip()

# --- 4.3. Eksekusi Contoh Inferensi ---
# PENTING: Karena training hanya 1 epoch, hasil terjemahan
# kemungkinan besar masih akan BURUK.
# Namun, dengan dataset kecil ini, hasilnya mungkin sedikit lebih baik
# daripada model OOM Anda sebelumnya.

print("\n--- Menjalankan Contoh Terjemahan ---")

# Ambil beberapa contoh dari data validasi untuk perbandingan
print("\nContoh 1 (dari data validasi):")
src_text_1 = val_df.iloc[0]['en']
tgt_text_1 = val_df.iloc[0]['fr'] # <-- BERUBAH: 'fr'
translation_1 = translate(model, src_text_1, vocab_src, vocab_tgt, tokenize_en)
print(f"  EN (Source): {src_text_1}")
print(f"  FR (Target): {tgt_text_1}")  # <-- BERUBAH: FR
print(f"  FR (Model) : {translation_1}")  # <-- BERUBAH: FR


print("\nContoh 2 (dari data validasi):")
src_text_2 = val_df.iloc[1]['en']
tgt_text_2 = val_df.iloc[1]['fr'] # <-- BERUBAH: 'fr'
translation_2 = translate(model, src_text_2, vocab_src, vocab_tgt, tokenize_en)
print(f"  EN (Source): {src_text_2}")
print(f"  FR (Target): {tgt_text_2}")  # <-- BERUBAH: FR
print(f"  FR (Model) : {translation_2}")  # <-- BERUBAH: FR

print("\nContoh 3 (Input Kustom):")
src_text_3 = "he is never cold in december ."
translation_3 = translate(model, src_text_3, vocab_src, vocab_tgt, tokenize_en)
print(f"  EN (Source): {src_text_3}")
print(f"  FR (Model) : {translation_3}")  # <-- BERUBAH: FR

print("\n--- Selesai Fokus Penilaian 4 ---")


--- Memulai Fokus Penilaian 4: Inferensi ---

--- Menjalankan Contoh Terjemahan ---

Contoh 1 (dari data validasi):
  EN (Source): the mango is their most loved fruit , but the apple is my most loved .
  FR (Target): la mangue est leur fruit le plus cher , mais la pomme est mon plus aimé .
  FR (Model) : la mangue est leur fruit le plus cher , mais la pomme est mon plus aimé .

Contoh 2 (dari data validasi):
  EN (Source): china is sometimes busy during july , and it is sometimes rainy in march .
  FR (Target): la chine est parfois occupé en juillet , et il est parfois pluvieux en mars .
  FR (Model) : chine est parfois occupé en juillet , et il est parfois pluvieux en mars .

Contoh 3 (Input Kustom):
  EN (Source): he is never cold in december .
  FR (Model) : il ne fait jamais froid en décembre .

--- Selesai Fokus Penilaian 4 ---
